<h2>Wrapper combining OpenAI, Melissa, Yelp</h2>

In [23]:
pip install xmltodict openai

Note: you may need to restart the kernel to use updated packages.Requirement already satisfied: xmltodict in c:\python3.11\lib\site-packages (0.13.0)




[notice] A new release of pip is available: 23.2.1 -> 23.3.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [24]:
pip install python-dotenv

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.2.1 -> 23.3.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [25]:
import urllib.request, urllib.parse, os, json, xmltodict, asyncio
from openai import AsyncOpenAI
from dotenv import load_dotenv
load_dotenv()


True

In [26]:
async def api_wrapper(location, preferences = ""):
    
    # Get a list of 10 locations form OpenAI api
    openai_client = AsyncOpenAI(api_key=os.environ.get("OPENAI_API_KEY"))
    chat_completion = await openai_client.chat.completions.create(
        model="gpt-4-turbo-preview",
        response_format={ "type": "json_object" },
        messages=[
            {"role": "system", "content": "You are a helpful travel assistant designed to output JSON containing a list of objects that have atrribute name and address."},
            {"role": "user", "content": f"I am traveling to {location}, and I want to visit attractions around this area that fit these criteria: {preferences}. Generate a list of 10 attraction objects and return each attraction name and entire address"}
        ],
    )
    openai_locations = json.loads(chat_completion.choices[0].message.content)["attractions"]
    
    # Get a list of 10 locations from YelpAPI
    term = preferences if preferences else "attractions"
    radius = 20000
    sort_by = "best_match"
    limit = 20 # Just in case 10 duplicates
    yelp_url = "https://api.yelp.com/v3/businesses/search?" + urllib.parse.urlencode([("location", location), ("term", term), ("radius", radius), ("sort_by", sort_by), ("limit", limit)])
    yelp_header = {
    "accept": "application/json",
    "Authorization": "Bearer " + os.getenv("YELP_API_KEY")
    }
    yelp_request = urllib.request.Request(yelp_url, headers = yelp_header)
    yelp_response = urllib.request.urlopen(yelp_request)
    if yelp_response.getcode() == 200:
        yelp_locations = json.loads(yelp_response.read().decode("utf-8"))
    
    # # Add melissa for validating hotels later
    return {"OpenAI": openai_locations, "Yelp": yelp_locations}

In [27]:
await api_wrapper("American High School, Fremont", "technology museum")

{'OpenAI': [{'name': 'The Tech Interactive',
   'address': '201 S Market St, San Jose, CA 95113, United States'},
  {'name': 'Computer History Museum',
   'address': '1401 N Shoreline Blvd, Mountain View, CA 94043, United States'},
  {'name': 'Intel Museum',
   'address': '2200 Mission College Blvd, Santa Clara, CA 95054, United States'},
  {'name': 'NASA Ames Visitor Center',
   'address': 'Mail Stop 207-3, Moffett Field, CA 94035, United States'},
  {'name': 'Exploratorium',
   'address': 'Pier 15, The Embarcadero, San Francisco, CA 94111, United States'},
  {'name': 'Museum of American Heritage',
   'address': '351 Homer Ave, Palo Alto, CA 94301, United States'},
  {'name': 'Bay Area Discovery Museum',
   'address': '557 McReynolds Rd, Sausalito, CA 94965, United States'},
  {'name': 'Lawrence Hall of Science',
   'address': '1 Centennial Dr, Berkeley, CA 94720, United States'},
  {'name': 'Autodesk Gallery',
   'address': 'One Market, 2nd Floor, San Francisco, CA 94105, United Stat